In [1]:
"""
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # ERROR 메시지만 출력하도록 설정
# GPU 설정 로그 억제
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
"""

'\nimport os\nos.environ[\'TF_CPP_MIN_LOG_LEVEL\'] = \'3\'  # ERROR 메시지만 출력하도록 설정\n# GPU 설정 로그 억제\nimport tensorflow as tf\ntf.get_logger().setLevel(\'ERROR\')\nos.environ["CUDA_VISIBLE_DEVICES"] = "0"\n'

In [2]:
# 설정 변수
epochs = 20
batch_size = 32
validation_split = 0.2

In [3]:
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# 파일 경로 설정
train_file_path = 'data/mnt/train.csv'
export_path = 'data/'
train_data = pd.read_csv(train_file_path)

# 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # 다중 공백 제거
    text = re.sub(r'[^\w\s]', '', text)  # 특수 문자 제거
    return text.strip()

train_data['cleaned_conversation'] = train_data['conversation'].apply(clean_text)
train_data['text_length'] = train_data['cleaned_conversation'].apply(len)

# 토크나이저 설정
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['cleaned_conversation'])

# 시퀀스 변환 및 패딩
sequences = tokenizer.texts_to_sequences(train_data['cleaned_conversation'])
max_length = 300
X_data = pad_sequences(sequences, maxlen=max_length, padding='post')

# 레이블 인코딩
label_encoder = LabelEncoder()
y_data = label_encoder.fit_transform(train_data['class'])

# 결과 확인
print('토크나이저 단어 개수:', len(tokenizer.word_index))
print('X_data 크기:', X_data.shape)
print('y_data 크기:', y_data.shape)

토크나이저 단어 개수: 52568
X_data 크기: (3950, 300)
y_data 크기: (3950,)


In [4]:
# 전처리된 데이터 저장
# np.save(export_path + 'X_data.npy', X_data)
# np.save(export_path + 'y_data.npy', y_data)

# 데이터 로드
# padded_sequences = np.load('data/X_data.npy')
#labels = np.load('data/y_data.npy')
padded_sequences = X_data
labels = y_data

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

# 입력 정의
input_layer = Input(shape=(max_length,), name='input')

# 임베딩 층
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)(input_layer)

# LSTM 층
lstm_layer = LSTM(64)(embedding_layer)

# 출력층 정의
num_classes = 4
output_layer = Dense(num_classes, activation='softmax', name='output')(lstm_layer)

# 모델 생성
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 300)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 128)          6728832   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
output (Dense)               (None, 4)                 260       
Total params: 6,778,500
Trainable params: 6,778,500
Non-trainable params: 0
_________________________________________________________________


In [6]:
# 모델 학습
history = model.fit(
    padded_sequences, labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=validation_split
)

Epoch 1/20
99/99 [==============================] - 29s 19ms/step - loss: 1.3845 - accuracy: 0.2744 - val_loss: 1.3928 - val_accuracy: 0.2494
Epoch 2/20
99/99 [==============================] - 1s 14ms/step - loss: 1.3829 - accuracy: 0.2839 - val_loss: 1.3926 - val_accuracy: 0.2494
Epoch 3/20
99/99 [==============================] - 1s 14ms/step - loss: 1.3836 - accuracy: 0.2756 - val_loss: 1.3890 - val_accuracy: 0.2494
Epoch 4/20
99/99 [==============================] - 1s 14ms/step - loss: 1.3833 - accuracy: 0.2839 - val_loss: 1.3929 - val_accuracy: 0.2494
Epoch 5/20
99/99 [==============================] - 1s 14ms/step - loss: 1.3833 - accuracy: 0.2839 - val_loss: 1.3945 - val_accuracy: 0.2494
Epoch 6/20
99/99 [==============================] - 1s 14ms/step - loss: 1.3828 - accuracy: 0.2839 - val_loss: 1.3906 - val_accuracy: 0.2494
Epoch 7/20
99/99 [==============================] - 1s 14ms/step - loss: 1.3831 - accuracy: 0.2839 - val_loss: 1.3904 - val_accuracy: 0.2494
Epoch 8/20
9

# 1. CNN 구조를 엮은 모델

In [7]:
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense

input_layer = Input(shape=(max_length,), name='input')

embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)(input_layer)

conv_layer = Conv1D(filters=128, kernel_size=5, activation='relu')(embedding_layer)
polling_layer = GlobalMaxPooling1D()(conv_layer)

output_layer = Dense(num_classes, activation='softmax', name='output')(polling_layer)

model_conv = Model(inputs=input_layer, outputs=output_layer)
model_conv.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_conv.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 300)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 128)          6728832   
_________________________________________________________________
conv1d (Conv1D)              (None, 296, 128)          82048     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
output (Dense)               (None, 4)                 516       
Total params: 6,811,396
Trainable params: 6,811,396
Non-trainable params: 0
_________________________________________________________________


In [8]:
con_history = model.fit(
    padded_sequences,
    labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=validation_split
)

Epoch 1/20
99/99 [==============================] - 1s 14ms/step - loss: 1.3825 - accuracy: 0.2839 - val_loss: 1.3904 - val_accuracy: 0.2494
Epoch 2/20
99/99 [==============================] - 1s 14ms/step - loss: 1.3826 - accuracy: 0.2839 - val_loss: 1.3917 - val_accuracy: 0.2494
Epoch 3/20
99/99 [==============================] - 1s 14ms/step - loss: 1.3827 - accuracy: 0.2839 - val_loss: 1.3913 - val_accuracy: 0.2494
Epoch 4/20
99/99 [==============================] - 1s 14ms/step - loss: 1.3825 - accuracy: 0.2839 - val_loss: 1.3917 - val_accuracy: 0.2494
Epoch 5/20
99/99 [==============================] - 1s 14ms/step - loss: 1.3825 - accuracy: 0.2839 - val_loss: 1.3908 - val_accuracy: 0.2494
Epoch 6/20
99/99 [==============================] - 1s 14ms/step - loss: 1.3825 - accuracy: 0.2839 - val_loss: 1.3916 - val_accuracy: 0.2494
Epoch 7/20
99/99 [==============================] - 1s 14ms/step - loss: 1.3825 - accuracy: 0.2839 - val_loss: 1.3907 - val_accuracy: 0.2494
Epoch 8/20
99

# Bidirectional LSTM

In [9]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense

# 입력 정의
input_layer = Input(shape=(max_length,), name='input')

# 임베딩 층
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)(input_layer)

# 양방향 LSTM 층
bi_lstm_layer = Bidirectional(LSTM(64))(embedding_layer)

# 출력층 정의
output_layer = Dense(num_classes, activation='softmax', name='output')(bi_lstm_layer)

# 모델 생성
model_bi = Model(inputs=input_layer, outputs=output_layer)

# 모델 컴파일
model_bi.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
model_bi.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 300)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 300, 128)          6728832   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               98816     
_________________________________________________________________
output (Dense)               (None, 4)                 516       
Total params: 6,828,164
Trainable params: 6,828,164
Non-trainable params: 0
_________________________________________________________________


In [10]:
model_bi_history = model_bi.fit(
    padded_sequences,
    labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=validation_split
)

Epoch 1/20
99/99 [==============================] - 5s 30ms/step - loss: 1.2494 - accuracy: 0.4684 - val_loss: 1.0037 - val_accuracy: 0.5797
Epoch 2/20
99/99 [==============================] - 2s 23ms/step - loss: 0.6210 - accuracy: 0.7978 - val_loss: 0.5802 - val_accuracy: 0.8000
Epoch 3/20
99/99 [==============================] - 2s 23ms/step - loss: 0.1219 - accuracy: 0.9693 - val_loss: 0.5592 - val_accuracy: 0.8051
Epoch 4/20
99/99 [==============================] - 2s 23ms/step - loss: 0.0413 - accuracy: 0.9911 - val_loss: 0.6075 - val_accuracy: 0.8114
Epoch 5/20
99/99 [==============================] - 2s 23ms/step - loss: 0.0068 - accuracy: 0.9997 - val_loss: 0.6071 - val_accuracy: 0.8316
Epoch 6/20
99/99 [==============================] - 2s 23ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.6309 - val_accuracy: 0.8253
Epoch 7/20
99/99 [==============================] - 2s 23ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.6925 - val_accuracy: 0.8278
Epoch 8/20
99

# BERT 모델

In [11]:
"""
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D

bert_model_name = 'distilbert-base-uncased'

# BERT 모델 로드
bert_model = TFBertModel.from_pretrained(bert_model_name)

# BERT 토크나이저
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

# 입력 정의
input_ids = Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask = Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

# BERT 모델 레이어
bert_output = bert_model([input_ids, attention_mask])[0]

pooling_layer = GlobalAveragePooling1D()(bert_output)

# 출력층 정의
output_layer = Dense(num_classes, activation='softmax', name='output')(pooling_layer)

# 모델 생성
model_bert = Model(inputs=[input_ids, attention_mask], outputs=output_layer)

# 모델 컴파일
model_bert.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
model_bert.summary()
"""

"\nfrom transformers import BertTokenizer, TFBertModel\nfrom tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D\n\nbert_model_name = 'distilbert-base-uncased'\n\n# BERT 모델 로드\nbert_model = TFBertModel.from_pretrained(bert_model_name)\n\n# BERT 토크나이저\ntokenizer = BertTokenizer.from_pretrained(bert_model_name)\n\n# 입력 정의\ninput_ids = Input(shape=(max_length,), dtype=tf.int32, name='input_ids')\nattention_mask = Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')\n\n# BERT 모델 레이어\nbert_output = bert_model([input_ids, attention_mask])[0]\n\npooling_layer = GlobalAveragePooling1D()(bert_output)\n\n# 출력층 정의\noutput_layer = Dense(num_classes, activation='softmax', name='output')(pooling_layer)\n\n# 모델 생성\nmodel_bert = Model(inputs=[input_ids, attention_mask], outputs=output_layer)\n\n# 모델 컴파일\nmodel_bert.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])\n\n# 모델 요약\nmodel_bert.summary()\n"

In [12]:
"""
# BERT 입력 데이터 생성
def encode_textx(texts, max_length):
    inputs = tokenizer(texts, max_length=max_length, padding='max_length', truncation=True, return_tensors='tf')
    return inputs['input_ids'], inputs['attention_mask']

texts = train_data['cleaned_conversation'].tolist()
input_ids, attention_mask = encode_textx(texts, max_length)

# 모델 학습
history_bert = model_bert.fit(
    [input_ids, attention_mask],
    labels,
    epochs=epochs,
    batch_size=8,
    validation_split=validation_split
)
"""

"\n# BERT 입력 데이터 생성\ndef encode_textx(texts, max_length):\n    inputs = tokenizer(texts, max_length=max_length, padding='max_length', truncation=True, return_tensors='tf')\n    return inputs['input_ids'], inputs['attention_mask']\n\ntexts = train_data['cleaned_conversation'].tolist()\ninput_ids, attention_mask = encode_textx(texts, max_length)\n\n# 모델 학습\nhistory_bert = model_bert.fit(\n    [input_ids, attention_mask],\n    labels,\n    epochs=epochs,\n    batch_size=8,\n    validation_split=validation_split\n)\n"

# 사전 학습 모델

In [17]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# 텐서플로우 메모리 제한 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_virtual_device_configuration(
                gpu,
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])  # 메모리 제한 설정
    except RuntimeError as e:
        print(e)

Virtual devices cannot be modified after being initialized


In [20]:
from transformers import TFBertModel, BertTokenizer
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# BERT 모델 로드
bert_model_name = 'distilbert-base-multilingual-cased'
bert_model = TFBertModel.from_pretrained(bert_model_name)
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

# 데이터 전처리 함수
def encode_texts(texts, max_length=128):
   inputs = tokenizer(texts, max_length=max_length, padding='max_length', truncation=True, return_tensors='tf')
   return inputs['input_ids'], inputs['attention_mask']

# 데이터 로드 및 전처리
texts = train_data['cleaned_conversation'].tolist()
input_ids, attention_mask = encode_texts(texts)

# 입력 정의
input_ids = Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_masks = Input(shape=(max_length,), dtype=tf.int32, name='attention_masks')

# BERT 모델을 사용하여 임베딩 추출
bert_output = bert_model([input_ids, attention_mask])[0][:, 0, :]

# 임베딩 저장
# np.save('data/mnt/targets.npy')

# 임베딩 로드
# bert_embeddings = np.load('data/mnt/targets.npy')
bert_embeddings = bert_output

# 입력 정의
input_layer = Input(shape=(bert_embeddings.shape[1],), name='input')

# 출력층 정의
output_layer = Dense(num_classes, activation='softmax', name='output')(input_layer)

# 모델 생성
model_bert = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
model_bert.summary()


You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


ResourceExhaustedError: OOM when allocating tensor with shape[119547,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:TruncatedNormal]

In [ ]:
# 모델 학습
history_bert = model_bert.fit(
    bert_embeddings,
    labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=validation_split
)

In [ ]:
import matplotlib.pyplot as plt
# 학습 결과 시각화

num_model = 4
plt_size = num_model + 1

plt.figure(figsize=(12, 5 * plt_size))

plt.subplot(plt_size, 2, 1)
plt.plot(history.history['loss'], 'b-', label='lstm_loss')
plt.plot(con_history.history['loss'], 'r--', label='con_loss')
plt.plot(model_bi_history.history['loss'], 'g-.', label='bi_loss')
plt.plot(history_bert.history['loss'], 'k:', label='bert_loss')
plt.plot()
plt.title('All_loss')
plt.legend()

plt.subplot(plt_size, 2, 2)
plt.plot(history.history['accuracy'], 'g-', label='lstm_accuracy')
plt.plot(con_history.history['accuracy'], 'k--', label='con_accuracy')
plt.title('All_accuracy')
plt.legend()

plt.subplot(plt_size, 2, 3)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.title('LSTM')
plt.legend()

plt.subplot(plt_size, 2, 4)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.title('LSTM')
plt.legend()

plt.subplot(plt_size, 2, 5)
plt.plot(con_history.history['loss'], 'b-', label='loss')
plt.plot(con_history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.title('Conv1D-LSTM')
plt.legend()

plt.subplot(plt_size, 2, 6)
plt.plot(con_history.history['accuracy'], 'g-', label='accuracy')
plt.plot(con_history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.title('Conv1D-LSTM')
plt.legend()

plt.subplot(plt_size, 2, 7)
plt.plot(model_bi_history.history['loss'], 'b-', label='loss')
plt.plot(model_bi_history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.title('Bidirectional-LSTM')
plt.legend()

plt.subplot(plt_size, 2, 8)
plt.plot(model_bi_history.history['accuracy'], 'g-', label='accuracy')
plt.plot(model_bi_history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.title('Bidirectional-LSTM')
plt.legend()

plt.subplot(plt_size, 2, 9)
plt.plot(history_bert.history['loss'], 'b-', label='loss')
plt.plot(history_bert.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.title('BERT')
plt.legend()

plt.subplot(plt_size, 2, 10)
plt.plot(history_bert.history['accuracy'], 'g-', label='accuracy')
plt.plot(history_bert.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.title('BERT')
plt.legend()

plt.show()